# US Census Data



In [1]:
# Dependencies
import pandas as pd, numpy as np
import requests
from pathlib import Path
from census import Census

In [4]:
# Retrieve census data for 2022 which is not available via API 
import urllib.request, json 
with urllib.request.urlopen("https://api.census.gov/data/2022/acs/acs5?get=B08006_014E,B01003_001E,NAME&for=tract:*&in=county:075&in=state:06&key=921e59d6bcaa21630b4f53e74b7a522a3502b8cb") as url:
    data = json.load(url)
    df = pd.DataFrame(data)
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
df = df[['NAME', 'B01003_001E', 'B08006_014E', 'state', 'county', 'tract']]
df['year'] = 2022
df['B01003_001E'] = df['B01003_001E'].astype(float)
df['B08006_014E'] = df['B08006_014E'].astype(float)
census_2022 = df.copy()
census_2022

,NAME,B01003_001E,B08006_014E,state,county,tract,year
1,Census Tract 101.01; San Francisco County; Cal...,2121.0,27.0,06,075,010101,2022
2,Census Tract 101.02; San Francisco County; Cal...,1908.0,17.0,06,075,010102,2022
3,Census Tract 102.01; San Francisco County; Cal...,2473.0,30.0,06,075,010201,2022
4,Census Tract 102.02; San Francisco County; Cal...,1859.0,80.0,06,075,010202,2022
5,Census Tract 103; San Francisco County; Califo...,3925.0,40.0,06,075,010300,2022
...,...,...,...,...,...,...,...
240,Census Tract 9805.01; San Francisco County; Ca...,129.0,0.0,06,075,980501,2022
241,Census Tract 9806; San Francisco County; Calif...,1172.0,6.0,06,075,980600,2022
242,Census Tract 9809; San Francisco County; Calif...,315.0,11.0,06,075,980900,2022
243,Census Tract 9901; San Francisco County; Calif...,0.0,0.0,06,075,990100,2022


In [5]:
census_key = '921e59d6bcaa21630b4f53e74b7a522a3502b8cb'

# Define function to retrieve US census data for a specific year and return a dataframe
def get_census_data(year):
    census_library = Census(census_key, year=year)
    state_code = '06'
    county_code = '075'
    variables = [
        'NAME',
        'B01003_001E',   # Population
        'B08006_014E',   # Bicycle commuters
    ]
    
    # Retrieve data for all states
    data = census_library.acs5.state_county_tract(variables, state_code, county_code, Census.ALL)

    # Convert to dataframe
    df = pd.DataFrame(data)
    df['year'] = year

    # Return the dataframe
    return df

In [13]:
# Retrieve census data for multiple years and merge into one dataframe
first_year = 2012
last_year = 2021

# Code reference: https://blog.finxter.com/how-to-create-a-python-list-of-size-n/
year_df = [None] * (last_year - first_year + 1)
for year in range(first_year, last_year + 1):
    print(f'Getting data for {year}')
    year_df[year - first_year] = get_census_data(year)

# Code reference: https://www.geeksforgeeks.org/merge-two-dataframes-with-same-column-names/
census_df = pd.concat(year_df, axis=0)

Getting data for 2012
Getting data for 2013
Getting data for 2014
Getting data for 2015
Getting data for 2016
Getting data for 2017
Getting data for 2018
Getting data for 2019
Getting data for 2020
Getting data for 2021


In [14]:
# Use to merge list with 2022 dataframe
census_df = pd.concat([census_df, df])
census_df

,NAME,B01003_001E,B08006_014E,state,county,tract,year
0,"Census Tract 328.02, San Francisco County, Cal...",3787.0,20.0,06,075,032802,2012
1,"Census Tract 329.01, San Francisco County, Cal...",5606.0,0.0,06,075,032901,2012
2,"Census Tract 329.02, San Francisco County, Cal...",3691.0,0.0,06,075,032902,2012
3,"Census Tract 332.03, San Francisco County, Cal...",3845.0,0.0,06,075,033203,2012
4,"Census Tract 332.04, San Francisco County, Cal...",3830.0,97.0,06,075,033204,2012
...,...,...,...,...,...,...,...
240,Census Tract 9805.01; San Francisco County; Ca...,129.0,0.0,06,075,980501,2022
241,Census Tract 9806; San Francisco County; Calif...,1172.0,6.0,06,075,980600,2022
242,Census Tract 9809; San Francisco County; Calif...,315.0,11.0,06,075,980900,2022
243,Census Tract 9901; San Francisco County; Calif...,0.0,0.0,06,075,990100,2022


In [15]:
print(census_df.dtypes)
census_df

NAME            object
B01003_001E    float64
B08006_014E    float64
state           object
county          object
tract           object
year             int64
dtype: object


,NAME,B01003_001E,B08006_014E,state,county,tract,year
0,"Census Tract 328.02, San Francisco County, Cal...",3787.0,20.0,06,075,032802,2012
1,"Census Tract 329.01, San Francisco County, Cal...",5606.0,0.0,06,075,032901,2012
2,"Census Tract 329.02, San Francisco County, Cal...",3691.0,0.0,06,075,032902,2012
3,"Census Tract 332.03, San Francisco County, Cal...",3845.0,0.0,06,075,033203,2012
4,"Census Tract 332.04, San Francisco County, Cal...",3830.0,97.0,06,075,033204,2012
...,...,...,...,...,...,...,...
240,Census Tract 9805.01; San Francisco County; Ca...,129.0,0.0,06,075,980501,2022
241,Census Tract 9806; San Francisco County; Calif...,1172.0,6.0,06,075,980600,2022
242,Census Tract 9809; San Francisco County; Calif...,315.0,11.0,06,075,980900,2022
243,Census Tract 9901; San Francisco County; Calif...,0.0,0.0,06,075,990100,2022


In [16]:
census_df.drop(columns=['NAME', 'state', 'county'], inplace=True)
census_df.rename(columns={'B01003_001E': 'population', 'B08006_014E': 'bike_commuters'}, inplace=True)
census_df

,population,bike_commuters,tract,year
0,3787.0,20.0,032802,2012
1,5606.0,0.0,032901,2012
2,3691.0,0.0,032902,2012
3,3845.0,0.0,033203,2012
4,3830.0,97.0,033204,2012
...,...,...,...,...
240,129.0,0.0,980501,2022
241,1172.0,6.0,980600,2022
242,315.0,11.0,980900,2022
243,0.0,0.0,990100,2022


In [17]:
# Convert columns from float to int
census_df['population'] = census_df['population'].astype(int)
census_df['bike_commuters'] = census_df['bike_commuters'].astype(int)
census_df = census_df[['year', 'population', 'bike_commuters', 'tract']]
census_df

,year,population,bike_commuters,tract
0,2012,3787,20,032802
1,2012,5606,0,032901
2,2012,3691,0,032902
3,2012,3845,0,033203
4,2012,3830,97,033204
...,...,...,...,...
240,2022,129,0,980501
241,2022,1172,6,980600
242,2022,315,11,980900
243,2022,0,0,990100


In [18]:
# Export dataframe to csv 
census_df.to_csv('resources/census_data_tracts.csv')

# Confirm that export completed
print('Dataframe exported to csv')

Dataframe exported to csv
